In [94]:
import pandas as pd

matches = pd.read_csv('2023Scores.csv')
matches.head()

,Week,Date,Team,Score,Opponent,OpponentScore,HomeorAway
0,Week 1,09/07/2023,Detroit Lions,21,Kansas City Chiefs,20,Away
1,Week 2,09/17/2023,Detroit Lions,31,Seattle Seahawks,37,Home
2,Week 3,09/24/2023,Detroit Lions,20,Atlanta Falcons,6,Home
3,Week 4,09/28/2023,Detroit Lions,34,Green Bay Packers,20,Away
4,Week 1,09/10/2023,Tampa Bay Buccaneers,20,Minnesota Vikings,17,Away


In [95]:
#Turn Weeks into a integer so RFC can read it
matches['Week'] = matches['Week'].apply(lambda x: int(x.split()[1]))

#Create predictors and the target
matches['HomeCode'] = matches['HomeorAway'].astype("category").cat.codes
matches['opponentNum'] = matches['Opponent'].astype("category").cat.codes
matches['result'] = 0
matches['result'] = matches.apply(lambda row: 'W' if row['Score'] > row['OpponentScore'] else 'L', axis=1)
matches.head()
matches['target'] = (matches['result'] =="W").astype("int")
matches.head()

,Week,Date,Team,Score,Opponent,OpponentScore,HomeorAway,HomeCode,opponentNum,result,target
0,1,09/07/2023,Detroit Lions,21,Kansas City Chiefs,20,Away,0,15,W,1
1,2,09/17/2023,Detroit Lions,31,Seattle Seahawks,37,Home,1,28,L,0
2,3,09/24/2023,Detroit Lions,20,Atlanta Falcons,6,Home,1,1,W,1
3,4,09/28/2023,Detroit Lions,34,Green Bay Packers,20,Away,0,11,W,1
4,1,09/10/2023,Tampa Bay Buccaneers,20,Minnesota Vikings,17,Away,0,20,W,1


In [96]:
#import RFC, train RFC on data from weeks 1-3, and predict week 4 using opponent and if team was home or not
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50, min_samples_split=5, random_state=1)
trainData = matches[matches['Week'] <= 3]
testData = matches[matches['Week'] == 4]
predictors = ['opponentNum', 'HomeCode',]
rfc.fit(trainData[predictors], trainData['target'])


RandomForestClassifier(min_samples_split=5, n_estimators=50, random_state=1)

In [97]:
#import accuracy_score to see how accurate RFC was with the 2 predictors
preds = rfc.predict(testData[predictors])
from sklearn.metrics import accuracy_score
acc = accuracy_score(testData['target'], preds)
acc


0.53125

In [98]:
#import precision and see how precise preds were
from sklearn.metrics import precision_score
precision_score(testData['target'], preds)

0.5384615384615384

In [99]:
#Begin to create groups of teams in order to determine moving averages and add them as a predictor
grouped_matches = matches.groupby('Team')
group = grouped_matches.get_group("Detroit Lions")
group

,Week,Date,Team,Score,Opponent,OpponentScore,HomeorAway,HomeCode,opponentNum,result,target
0,1,09/07/2023,Detroit Lions,21,Kansas City Chiefs,20,Away,0,15,W,1
1,2,09/17/2023,Detroit Lions,31,Seattle Seahawks,37,Home,1,28,L,0
2,3,09/24/2023,Detroit Lions,20,Atlanta Falcons,6,Home,1,1,W,1
3,4,09/28/2023,Detroit Lions,34,Green Bay Packers,20,Away,0,11,W,1


In [100]:
#Gather mean of a column from previous 2 weeks and attach to third week data
def movingAverage(group, cols, new_cols):
  group = group.sort_values('Week')
  rolling_stats = group[cols].rolling(2, closed='left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group
#Take points, and points allowed by each team, determine average and rename.
cols = ['Score', 'OpponentScore']
new_cols = [f'{c}moving' for c in cols]

movingAverage(group, cols, new_cols)

,Week,Date,Team,Score,Opponent,OpponentScore,HomeorAway,HomeCode,opponentNum,result,target,Scoremoving,OpponentScoremoving
2,3,09/24/2023,Detroit Lions,20,Atlanta Falcons,6,Home,1,1,W,1,26.0,28.5
3,4,09/28/2023,Detroit Lions,34,Green Bay Packers,20,Away,0,11,W,1,25.5,21.5


In [101]:
#apply movingAverages to every team
matchesAverage = matches.groupby('Team').apply(lambda x: movingAverage(x, cols, new_cols))
matchesAverage

Week        Date                   Team  Score  \
Team                                                                        
Arizona Cardinals     26      3  09/24/2023      Arizona Cardinals     28   
                      27      4  10/01/2023      Arizona Cardinals     16   
Atlanta Falcons       122     3  09/24/2023        Atlanta Falcons      6   
                      123     4  10/01/2023        Atlanta Falcons      7   
Baltimore Ravens      114     3  09/24/2023       Baltimore Ravens     19   
...                         ...         ...                    ...    ...   
Tampa Bay Buccaneers  7       4  10/01/2023   Tampa Bay Buccaneers     26   
Tennessee Titans      10      3  09/24/2023       Tennessee Titans      3   
                      11      4  10/01/2023       Tennessee Titans     27   
Washington Commanders 86      3  09/24/2023  Washington Commanders      3   
                      87      4  10/01/2023  Washington Commanders     31   

                                       Opponent  OpponentScore HomeorAway  \
Team                                                                        
Arizona Cardinals     26         Dallas Cowboys             16       Home   
                      27    San Francisco 49ers             35       Away   
Atlanta Falcons       122         Detroit Lions             20       Away   
                      123  Jacksonville Jaguars             23       Away   
Baltimore Ravens      114    Indianapolis Colts             22       Home   
...                                         ...            ...        ...   
Tampa Bay Buccaneers  7      New Orleans Saints              9       Away   
Tennessee Titans      10       Cleveland Browns             27       Away   
                      11     Cincinnati Bengals              3       Home   
Washington Commanders 86          Buffalo Bills             37       Home   
                      87    Philadelphia Eagles             34       Away   

                           HomeCode  opponentNum result  target  Scoremoving  \
Team                                                                           
Arizona Cardinals     26          1            8      W       1         22.0   
                      27          0           27      L       0         28.0   
Atlanta Falcons       122         0           10      L       0         24.5   
                      123         0           14      L       0         15.5   
Baltimore Ravens      114         1           13      L       0         26.0   
...                             ...          ...    ...     ...          ...   
Tampa Bay Buccaneers  7           0           22      W       1         19.0   
Tennessee Titans      10          0            7      L       0         21.0   
                      11          1            6      W       1         15.0   
Washington Commanders 86          1            3      L       0         27.5   
                      87          0           25      L       0         19.0   

                           OpponentScoremoving  
Team                                            
Arizona Cardinals     26                  25.5  
                      27                  23.5  
Atlanta Falcons       122                 17.0  
                      123                 22.0  
Baltimore Ravens      114                 16.5  
...                                        ...  
Tampa Bay Buccaneers  7                   21.0  
Tennessee Titans      10                  20.0  
                      11                  25.5  
Washington Commanders 86                  24.5  
                      87                  35.0  

[64 rows x 13 columns]

In [102]:
#Clean up
matchesAverage = matchesAverage.droplevel('Team')
matchesAverage.index = range(matchesAverage.shape[0])
matchesAverage

,Week,Date,Team,Score,Opponent,OpponentScore,HomeorAway,HomeCode,opponentNum,result,target,Scoremoving,OpponentScoremoving
0,3,09/24/2023,Arizona Cardinals,28,Dallas Cowboys,16,Home,1,8,W,1,22.0,25.5
1,4,10/01/2023,Arizona Cardinals,16,San Francisco 49ers,35,Away,0,27,L,0,28.0,23.5
2,3,09/24/2023,Atlanta Falcons,6,Detroit Lions,20,Away,0,10,L,0,24.5,17.0
3,4,10/01/2023,Atlanta Falcons,7,Jacksonville Jaguars,23,Away,0,14,L,0,15.5,22.0
4,3,09/24/2023,Baltimore Ravens,19,Indianapolis Colts,22,Home,1,13,L,0,26.0,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,4,10/01/2023,Tampa Bay Buccaneers,26,New Orleans Saints,9,Away,0,22,W,1,19.0,21.0
60,3,09/24/2023,Tennessee Titans,3,Cleveland Browns,27,Away,0,7,L,0,21.0,20.0
61,4,10/01/2023,Tennessee Titans,27,Cincinnati Bengals,3,Home,1,6,W,1,15.0,25.5
62,3,09/24/2023,Washington Commanders,3,Buffalo Bills,37,Home,1,3,L,0,27.5,24.5


In [103]:
#Train RFC with new movingAverages predictors on week 4 matchups
#and return a dataframe with results and a precision percent
def make_predictions(data, predictors):
  trainData = data[data['Week'] < 4]
  testData = data[data['Week'] == 4]
  rfc.fit(trainData[predictors], trainData['target'])
  preds = rfc.predict(testData[predictors])
  combined = pd.DataFrame(dict(actual=testData['target'], predicted=preds), index=testData.index)
  precision = precision_score(testData['target'], preds)
  return combined, precision

combined, precision = make_predictions(matchesAverage, predictors + new_cols)
precision

#precision went up ~0.2307

0.7692307692307693

In [104]:
#Create a dataframe that shows the actual target and the RFC predicted targer with team names
def make_predictions_all_teams(data, predictors):
  predictions = []
  for team in data['Team'].unique():
    team_data = data[data['Team'] == team]
    trainData = team_data[team_data['Week'] < 4]
    testData = team_data[team_data['Week'] == 4]
    if not testData.empty:
      rfc.fit(trainData[predictors], trainData['target'])
      preds = rfc.predict(testData[predictors])
      combined = pd.DataFrame(dict(Team=team, actual=testData['target'], predicted=preds), index=testData.index)
      predictions.append(combined)

  return pd.concat(predictions)

all_team_predictions = make_predictions_all_teams(matchesAverage, predictors + new_cols)
all_team_predictions

,Team,actual,predicted
1,Arizona Cardinals,0,1
3,Atlanta Falcons,0,0
5,Baltimore Ravens,1,0
7,Buffalo Bills,1,1
9,Carolina Panthers,0,0
11,Chicago Bears,0,0
13,Cincinnati Bengals,0,1
15,Cleveland Browns,0,1
17,Dallas Cowboys,1,0
19,Denver Broncos,1,0
